# SVM (Linear Kernel) with Tuning

This notebook implements a Support Vector Machine (SVM) model with a Linear kernel. It includes:
1. Data Loading and Preprocessing
2. Feature Engineering (K-Means)
3. Bayesian Hyperparameter Optimization (Tuning C parameter)
4. Model Training
5. Model Saving
6. Model Loading and Prediction on Test Data

In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report

# =====================================================
# CONFIGURATION
# =====================================================
TRAIN_PATH = "../../Dataset/train.csv"
TEST_PATH = "../../Dataset/test.csv"
MODEL_SAVE_PATH = "svm_linear_model.pkl"
SUBMISSION_PATH = "submission_svm.csv"
TARGET = "spend_category"
ID_COL = "trip_id"
N_CLUSTERS = 6
RANDOM_STATE = 42

In [ ]:
# =====================================================
# DATA LOADING AND PREPROCESSING FUNCTIONS
# =====================================================

def load_data():
    train_df = pd.read_csv(TRAIN_PATH)
    test_df = pd.read_csv(TEST_PATH)
    return train_df, test_df

def preprocess_data(df, is_train=True):
    df = df.copy()
    
    # 1. Clean strings
    for c in df.columns:
        if df[c].dtype == object:
            df[c] = df[c].astype(str).str.strip().str.rstrip(',')
            
    # 2. Binary Columns
    binary_cols = [
        "is_first_visit","intl_transport_included","accomodation_included",
        "food_included","domestic_transport_included","sightseeing_included",
        "guide_included","insurance_included"
    ]
    for c in binary_cols:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip().str.lower()
            df[c] = df[c].replace({"yes": 1, "no": 0}).fillna(0)
            df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0).astype(int)

    # 3. Numeric Count Columns
    numeric_count_cols = ["num_females","num_males","mainland_stay_nights","island_stay_nights"]
    for c in numeric_count_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    # 4. Ordinal Encoding
    def clean_str(x):
        x = str(x).strip().lower()
        if x in ["nan", "none", "null", ""]:
            return np.nan
        return x

    if "days_booked_before_trip" in df.columns:
        df["days_booked_before_trip_clean"] = df["days_booked_before_trip"].apply(clean_str)
        ordinal_days = {"1-7": 1, "8-14": 2, "15-30": 3, "31-60": 4, "61-90": 5, "90+": 6}
        df["days_booked_before_trip_ord"] = df["days_booked_before_trip_clean"].map(ordinal_days)
        mode_val = df["days_booked_before_trip_ord"].mode()[0] if not df["days_booked_before_trip_ord"].mode().empty else 1
        df["days_booked_before_trip_ord"] = df["days_booked_before_trip_ord"].fillna(mode_val).astype(int)

    if "total_trip_days" in df.columns:
        df["total_trip_days_clean"] = df["total_trip_days"].apply(clean_str)
        ordinal_trip = {"1-6": 1, "7-14": 2, "15-30": 3, "30+": 4}
        df["total_trip_days_ord"] = df["total_trip_days_clean"].map(ordinal_trip)
        mode_val = df["total_trip_days_ord"].mode()[0] if not df["total_trip_days_ord"].mode().empty else 1
        df["total_trip_days_ord"] = df["total_trip_days_ord"].fillna(mode_val).astype(int)

    if "has_special_requirements" in df.columns:
        df["has_special_req_bin"] = df["has_special_requirements"].astype(str).apply(
            lambda x: 0 if x.lower() in ["none", "", "nan"] else 1
        )

    # 5. Outlier Removal (Train Only)
    if is_train and TARGET in df.columns:
        df = df[df[TARGET].notnull()].reset_index(drop=True)
        df = df[df["num_females"] <= 10]
        df = df[df["num_males"] <= 10]
        df = df[df["mainland_stay_nights"] <= 90]
        df = df[df["island_stay_nights"] <= 60]
    
    # 6. Fill Categorical Missing Values
    categorical_cols = [
        "country","age_group","travel_companions","main_activity",
        "visit_purpose","tour_type","info_source","arrival_weather"
    ]
    for c in categorical_cols:
        if c in df.columns:
            mode_val = df[c].mode()[0] if not df[c].mode().empty else "Unknown"
            df[c] = df[c].fillna(mode_val)

    return df

In [ ]:
# =====================================================
# FEATURE ENGINEERING (K-MEANS)
# =====================================================

def add_kmeans_features(train_df, test_df, n_clusters=6):
    numeric_features = [
        "num_females", "num_males", "mainland_stay_nights", "island_stay_nights",
        "days_booked_before_trip_ord", "total_trip_days_ord"
    ]
    
    X_train_num = train_df[numeric_features].copy()
    X_test_num = test_df[numeric_features].copy()
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_num)
    X_test_scaled = scaler.transform(X_test_num)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=RANDOM_STATE, n_init=10)
    train_df["kmeans_cluster"] = kmeans.fit_predict(X_train_scaled)
    test_df["kmeans_cluster"] = kmeans.predict(X_test_scaled)
    
    return train_df, test_df

In [ ]:
# =====================================================
# DATA PREPARATION
# =====================================================

print("Loading and preprocessing data...")
train_raw, test_raw = load_data()
train_clean = preprocess_data(train_raw, is_train=True)
test_clean = preprocess_data(test_raw, is_train=False)

print(f"Adding K-Means features (K={N_CLUSTERS})...")
train_clean, test_clean = add_kmeans_features(train_clean, test_clean, n_clusters=N_CLUSTERS)

# Define Features
numeric_features = [
    "num_females", "num_males", "mainland_stay_nights", "island_stay_nights",
    "days_booked_before_trip_ord", "total_trip_days_ord"
]
binary_features = [
    "is_first_visit","intl_transport_included","accomodation_included",
    "food_included","domestic_transport_included","sightseeing_included",
    "guide_included","insurance_included", "has_special_req_bin"
]
categorical_features = [
    "country","age_group","travel_companions","main_activity",
    "visit_purpose","tour_type","info_source","arrival_weather",
    "kmeans_cluster"
]

all_features = numeric_features + binary_features + categorical_features

# Prepare X and y
X = train_clean[all_features]
y = train_clean[TARGET]

# Preprocessing Pipeline
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ("bin", "passthrough", binary_features)
])

# Split for Validation during Optimization
X_train_opt, X_val_opt, y_train_opt, y_val_opt = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

In [ ]:
# =====================================================
# BAYESIAN OPTIMIZATION (OPTUNA)
# =====================================================

def objective(trial):
    C = trial.suggest_float("C", 0.0001, 10.0, log=True)
    
    model = Pipeline([
        ("preprocessor", preprocessor),
        ("svm", SVC(
            kernel="linear",
            C=C,
            decision_function_shape="ovo",
            random_state=RANDOM_STATE
        ))
    ])
    
    model.fit(X_train_opt, y_train_opt)
    preds = model.predict(X_val_opt)
    acc = accuracy_score(y_val_opt, preds)
    
    return acc

print("Starting Bayesian Optimization...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10) # Reduced trials for speed

print("Best Params:", study.best_params)

In [ ]:
# =====================================================
# TRAIN FINAL MODEL
# =====================================================

best_C = study.best_params["C"]

final_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("svm", SVC(
        kernel="linear",
        C=best_C,
        decision_function_shape="ovo",
        random_state=RANDOM_STATE
    ))
])

print("Training final model on full training set...")
final_pipeline.fit(X, y)
print("Training complete.")

In [ ]:
# =====================================================
# SAVE MODEL
# =====================================================
print(f"Saving model to {MODEL_SAVE_PATH}...")
joblib.dump(final_pipeline, MODEL_SAVE_PATH)
print("Model saved successfully.")

In [ ]:
# =====================================================
# LOAD MODEL AND PREDICT
# =====================================================
print(f"Loading model from {MODEL_SAVE_PATH}...")
loaded_pipeline = joblib.load(MODEL_SAVE_PATH)

print("Predicting on test set...")
X_test = test_clean[all_features]
test_preds = loaded_pipeline.predict(X_test)

# Save Predictions
submission = pd.DataFrame({
    ID_COL: test_clean[ID_COL],
    TARGET: test_preds
})
submission.to_csv(SUBMISSION_PATH, index=False)
print(f"Predictions saved to {SUBMISSION_PATH}")

print(submission.head())